In [69]:
import re
from ufal.udpipe import Model, Pipeline, ProcessingError
import pandas as pd 
from conllu import parse

model = Model.load('G:/Softwares/urdu-udtb-ud-2.3-181115.udpipe')
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
error = ProcessingError()

## To get row count only

In [2]:
import pyodbc
from gensim.models import Word2Vec
import sys
import time
start = time.time()

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=G:\Softwares\EQWH-V6.4-190407\EasyHadees - Copy.mdb;')
cursor = conn.cursor()
targetcount = 100
cursor.execute("select count(*) from ahadith where urdutext is not null and urdutext like '%وضو%'")

count = cursor.fetchone()[0]
print(count)
cursor.close()
conn.close()

C:\Users\admin\Anaconda3\envs\islamic qa\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


6709


In [181]:
import pyodbc
from gensim.models import Word2Vec
import sys
import time
import numpy as np
start = time.time()

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=G:\Softwares\EQWH-V6.4-190407\EasyHadees - Copy.mdb;')
cursor = conn.cursor()
cursor.execute("select top 5 id, urdutext from ahadith where urdutext is not null and urdutext like '%وضو%'")

Y = []
corpus = []
lens_of_d= []
for row in cursor.fetchall():
    hadith = row[1].replace('صلی اللہ علیہ وآلہ وسلم','')    
    corpus.append(hadith)
    lens_of_d.append(len(hadith.split()))
    Y.append(row[0])
cursor.close()
conn.close()

avg_docs_len = np.mean(lens_of_d, axis=0)

In [182]:
from __future__ import unicode_literals
# -*- coding: utf-8 -*-
STOP_WORDS = """
 ابھی از 
 اس استعمال اسی اسے البتہ الف ان اندر اور اوپر اکثر اگر
 اگرچہ اگلے ایسا ایسی ایسے اے بار بارے باوجود باہر بظاہر بعض بغیر بلکہ بن
 بھر بھریں بھی بہت بے تا تاکہ تب تحت تر تمام
 تو تک جب سوال 
 جبکہ جو حالانکہ حالاں خلاف
 خود سے میں چنانچہ دیر ذریعے تھا هے ایک آپ
 سا ساتھ سامنے سب تھی تھے
 سکا سکتا سکتے سی سے شان شاید صرف صورت ضرورت ضروری طرح طرف طور علاوہ عین غیر 
 لہذا لیکن لیں لیے لے مجھ مجھے مزید مقابلے مل مکمل مگر 
 نا نہ نہیں نیچے واقعی والا والوں والی والے وجہ وغیرہ وہ وہاں وہی وہیں وی ویسے پاس
 پایا پر پوری پھر پیچھے چونکہ چکی
 ڈالے کئے کافی کبھی کسی کم 
 کوئی کچھ کہ کہا کہہ کہیں کہے کیونکہ کیے کے گئی
 گئے گا گویا گی گے ہاں ہر ہمیشہ ہو ہوئی ہوئیں ہوئے ہوا ہوتا
 ہوتی ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیں ہے یا یات یعنی یقینا یہ یہاں یہی یہیں ھیں
  حضرت ہوگی رض
 کا کو کی نے ہوتو پہلے کر
 اپنا حتى كہ ميں لئیے  
 بعد ارادہ افاقہ آخر
""".split()

In [183]:
#from sklearn.feature_extraction.text import TfidfVectorizer as vectorizer
from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = vectorizer()
#X = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())

vec = CountVectorizer(stop_words=STOP_WORDS).fit(corpus)
bag_of_words = vec.transform(corpus)
vocab = vec.vocabulary_.items()

In [184]:
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vocab]
def term_freq(myword):
    return ([tuple for tuple in words_freq if myword in tuple][0])[1]

In [185]:
def term_freq_doc(myword,doc_index):
    word_freq_doc = [(word, bag_of_words[doc_index].sum(axis=0)[0, idx]) for word, idx in vocab]
    if len(word_freq_doc) == 0:
        return 0
    term_tuple = [tuple for tuple in word_freq_doc if myword in tuple]
    if len(term_tuple) ==0:
        return 0
    return (term_tuple[0])[1]

In [186]:
#term_freq_doc('پانی',0)

# Ranking (okapi BM25)

![title](img/43e5c609557364f7836b6b2f4cd8ea41deb86a96.svg)

where

![title](img/c652b6871ce4872c8e924ff0f806bc8b06dc94ed.svg)

In [192]:
def no_of_docs(q):
    no_of_ds = len([di for di in range(0,len(Y)) if term_freq_doc(q,di) > 0])    
    return no_of_ds        

5

In [218]:
import math
def idf(q):
    nq = no_of_docs(q)
    ans = math.log10(  (len(Y)- nq + 0.5) / (nq+ 0.5)  )
    return ans

def bm25_score(qwords,doc_index):
    score = 0;
    for qw in qwords:
        tfd = term_freq_doc(qw, doc_index)
        score += idf(qw) * ( (tfd * (1.2 + 1))/ (tfd + 1.2 * (1-0.75 + 0.75 * (lens_of_d[doc_index] / avg_docs_len))))
    return score

In [219]:
from gensim.models import word2vec
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('urduvec_140M_100K_300d.bin', binary=True)

In [220]:
question = 'وضو کیسے کریں'
question_arr = question.split()

query_words = []
query_words.append(question_arr)
for question_word in question_arr:
    alltop = model.most_similar(positive=question_word, topn=20)
    for key, val in [item for item in alltop if item[1] > 0.51]:
        query_words.append(key)

In [224]:
score = bm25_score(query_words, 2)
print(score)

C:\Users\admin\Anaconda3\envs\islamic qa\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


-0.6807580442743947


In [226]:
corpus[0]

' ابوالنعمان، ابوعوانہ، ابی بشر، یوسف بن ماہک، عبداللہ بن عمرو (رض) کہتے ہیں کہ ایک سفر میں نبی () ہم سے پیچھے رہ گئے، جب آپ ہمارے قریب پہنچے تو نماز میں تاخیر ہونے (کی وجہ سے) ہم (جلد جلد) وضو کر رہے تھے، اسی وجہ سے ہم اپنے پیروں پر پانی ملنے لگے (کیونکہ دھونے میں دیر ہوتی) پس آپ نے اپنی بلند آواز سے دو یا تین مرتبہ فرمایا کہ (پیروں کے) ٹخنوں کو آگ کے (عذاب) سے خرابی (ہونے والی) ہے۔ '